<br>
    <center>МИНИСТЕРСТВО НАУКИ И ВЫСШЕГО ОБРАЗОВАНИЯ</center>
    <center>федеральное государственное автономное образовательное учреждение высшего образования </center> <center>«Самарский национальный исследовательский университет имени академика С.П. Королева»</center>
    <center>(Самарский университет)</center> </br>

<br/>

<br>
<center>Институт 	     информатики и кибернетики</center>                                                   	  
<center>Кафедра 	     технической кибернетики</center>                                                              	
</br>
<br/>

<br>

<br/>
<br/>
<br/>
<br>
<center>ОТЧЕТ</center>
<center>по лабораторной работе №1</center>

<center>«Введение в MapReduce модель на Python»</center>
<br/>
<center>по дисциплине <strong>«Большие данные»</strong></center>
<br/>
<center></center>
</br>
<br/>
<br/>
<br/>



<br>
<p style="text-align:right;">Выполнил: Васильев А.С.
<br>6131-010402D
<br>    
<br>Преподаватель: Попов С.Б.
</p>
<br/>
<br/>
<br/>
    <br/>
<br/>
<br/>
<center>Самара 2025</center>

# **Модель MapReduce**
Функция MapReduce имитирует работу фреймворка MapReduce: подготовка и форматирование исходных данных в виде набора пар ключ-значение (key-value) для последующего вызова функции MAP, вызов функции MAP, группировка промежуточных результатов работы функции MAP по ключу и формирование массива значений для каждого ключа, вызов функции REDUCE.

Пользователь для решения своей задачи реализует функции RECORDREADER, MAP, REDUCE.


In [1]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Режимы данных:
- `embedded` — использовать встроенные в ноутбук данные (аннотации и граф).
- `docx` — распарсить `/mnt/data/Lab1_данные.docx` напрямую без внешних библиотек.

Переключается константой `DATA_MODE` ниже.


In [2]:
from pathlib import Path

DATA_MODE = "docx"
LINE_MODE = False
DOCX_PATH = "Lab1_данные.docx"

print("CWD --> ", Path.cwd())

CWD -->  c:\Users\Артем\Desktop\Study\1 sem\BigData\1 lab


# Обработка документа

In [3]:
from zipfile import ZipFile
import xml.etree.ElementTree as ET
import re


def read_docx_file(path) -> list[str]:
    """ Чтение .docx файла и возврат как список строк """

    # открытие файла с помощью класса ZipFile и последующее чтение XML документа
    with ZipFile(path) as zf:
        xml = zf.read("word/document.xml")

    ns = {"w" : "http://schemas.openxmlformats.org/wordprocessingml/2006/main"}     # пространство имен WordprocessingML
    root = ET.fromstring(xml)       # парсим xml и собираем список параграфов

    paras = []
    for p in root.findall(".//w:p", ns):
        texts = [t.text for t in p.findall(".//w:t", ns) if t.text]     # внутри абзаца собираем все текстовые узлы <w:t>
        paras.append("".join(texts) if texts else "")       # склевиаем в одну строку 

    return [re.sub(r"\s+", " ", t).strip() for t in paras]      # нормализация пробелов в каждом параграфе - сжимаем подряд и обрезаем края

In [4]:
def parse_lab_data_from_docx(path):

    paras = read_docx_file(path)
    text = "\n".join(paras)

    # -------- Извлечение документов (аннотаций) -------- #

    docs, current = [], []
    in_docs = False

    for line in paras:

        # заголовок раздела аннотации 
        if re.search(r"^\s*Аннотац", line, re.IGNORECASE):

            # если накапливался предю документ - сохраняем
            if current:
                docs.append(" ".join(current).strip())
                current = []
            
            in_docs = True
            continue

        if in_docs:
            
            # дошли до раздела со списком смежности - значит, аннотация закончилась
            if re.search(r"Список\s+смежности", line, re.IGNORECASE):
                break
            
            # копим непустые строки текущего документа
            if line.strip():
                current.append(line.strip())

    # финальный "хвост" документа, если остался    
    if current:
        docs.append(" ".join(current).strip())


    # -------- Извлечение списка смежности -------- #

    adj = {}
    in_adj = False

    for line in paras:

        # начало раздела "Список смежности"
        if re.search(r"Список\s+смежности", line, re.IGNORECASE):
            in_adj = True
            continue

        if in_adj:

            # пропускаем строки без цифр
            if not re.search(r"\d", line):
                continue
            
            # достаем все числа из строки: первое - вершина, остальное соседи
            nums = list(map(int, re.findall(r"\d+", line)))
            if nums:
                u, *nrbs = nums
                adj[u] = nrbs


    return (docs, adj)

# Проверка работоспособности 

In [5]:
documents = []
adj = {}
if DATA_MODE == "docx":
    try:
        # Пытаемся распарсить DOCX: получаем корпус аннотаций и список смежности
        documents, adj = parse_lab_data_from_docx(DOCX_PATH)

        # Если парсер вернул пустые данные — мягко откатываемся на встроенные (embedded)
        if not documents or not adj:
            print("[WARN] DOCX parsing incomplete — fallback to embedded.")

            # Встроенный корпус документов (embedded), чтобы ноутбук был самодостаточным
            documents = ["Streaming data is the data from sensors as well as other real-time surveillance systems. Distributed stream processing systems are the software that manages such data. Such frameworks have to deliver outcomes on the go instantly. They are susceptible to delay and malfunction or system failures. The system must be tolerant of faults and always accessible. Many variables, such as improved network arrival rates, node failures, and so on, disrupt the system's reliability. Some operators need to be relocated online from one physical resource to another to manage or reimburse a slow or failing node. In this study, we propose a co-location based systematic migration heuristic for live operator migration between physical resources using a migration map revised with costs for each migration. The suggested method evaluates continuous operator performance patterns and makes online scheduling decisions based on the same. The decisions include migrating operators during a node failure or straggling.", "Distributed stream processing engines are designed with a focus on scalability to process big data volumes in a continuous manner. We present the Theodolite method for benchmarking the scalability of distributed stream processing engines. Core of this method is the definition of use cases that microservices implementing stream processing have to fulfill. For each use case, our method identifies relevant workload dimensions that might affect the scalability of a use case. We propose to design one benchmark per use case and relevant workload dimension. We present a general benchmarking framework, which can be applied to execute the individual benchmarks for a given use case and workload dimension. Our framework executes an implementation of the use case's dataflow architecture for different workloads of the given dimension and various numbers of processing instances. This way, it identifies how resources demand evolves with increasing workloads. Within the scope of this paper, we present 4 identified use cases, derived from processing Industrial Internet of Things data, and 7 corresponding workload dimensions. We provide implementations of 4 benchmarks with Kafka Streams and Apache Flink as well as an implementation of our benchmarking framework to execute scalability benchmarks in cloud environments. We use both for evaluating the Theodolite method and for benchmarking Kafka Streams' and Flink's scalability for different deployment options.", "Batch and stream processing are separately and efficiently applied in many applications. However, some newer data-driven applications such as the Internet of Things and cloud computing call for hybrid processing approaches in order to handle the speed and accuracy required for processing such complex data. In this paper, we propose a Hybrid Distributed Batch-Stream (HDBS) architecture for anomaly detection in real-time data. The hybrid architecture, while benefiting from the accuracy provided by batch processing, also enjoys the speed and real-time features of stream processing. In the proposed architecture, our focus is on the algorithmic aspects of hybrid processing including the interaction models between batch and stream processing units, the characteristics of batch and stream machine learning algorithms and the principles of merging the results of different processing units. The driving idea of such combination is that the results of batch and stream processing units are complementary with each other, as one of them constructs accurate models based on previous data, and the other one is capable of processing new stream data in real-time. Furthermore, we propose a generalized version of the HDBS with respect to its algorithms and communication policy levels. In the generalized HDBS architecture, we address the various aspects of the interaction between the batch and stream processing units, and the merging operations to produce the final results. the evaluations of the proposed architecture using various criteria (accuracy, space complexity, and time complexity) demonstrate that the accuracy of the proposed method is higher than the accuracy of the batch processing methods, its time complexity is also similar to one of the stream processing methods and much less than the batch processing methods, which makes our proposed architecture an efficient and practical solution for real-time anomaly detection.", "There have been increasing demands for real time processing of the ever-growing data. In order to meet this requirement and ensure the reliable processing of streaming data, a variety of distributed stream processing architectures and platforms have been developed, which handles the fundamental task of allocating processing tasks to the currently available physical resources and routing streaming data between these resources. However, many stream processing systems lack an intelligent scheduling mechanism, in which their default schedulers allocate tasks without taking resource demands and availability, or the transfer latency between resources into consideration. Besides, stream processing has a strict request for latency. Thus it is important to give latency guarantee for distributed stream processing. In this paper, we propose two new algorithms for stream processing with latency guarantee, both the algorithms consider transfer latency and resource demand in task allocation. Both algorithms can guarantee latency constraints. Algorithm AHA reduces more than 21.3% and 58.9% resources compared with the greedy and the round-robin algorithms, and algorithm PHA further improves the resource utilization to 32.1% and 73.2%.", "In the era of Big Data, typical architecture of distributed real-time stream processing systems is the combination of Flume, Kafka, and Storm. As a kind of distributed message system, Kafka has the characteristics of horizontal scalability and high throughput, which is manly deployed in many areas in order to address the problem of speed mismatch between message producers and consumers. When using Kafka, we need to quickly receive data sent by producers. In addition, we need to send data to consumers quickly. Therefore, the performance of Kafka is of critical importance to the performance of the whole stream processing system. In this paper, we propose the improved design of real-time stream processing systems, and focus on improving the Kafka’s data loading process. We use Kafka cat to transfer data from the source to Kafka topic directly, which can reduce the network transmission. We also utilize the memory file system to accelerate the process of data loading, which can address the bottleneck and performance problems caused by disk I/O. Extensive experiments are conducted to evaluate the performance, which show the superiority of our improved design.", "In this paper, nearly 40 commonly used deep neural network(DNN) models are selected, and their cross-platform and cross-inference frameworks are deeply analysed. The main metrics of accuracy, the total number of model parameters, the computational complexity, the accuracy density, the inference time, the memory consumption and other related parameters are used to measure their performance. The heterogeneous computing experiment is implemented on both the Google Colab cloud computing platform and the Jetson Nano embedded edge computing platform. The obtained performance is compared with that of two previous computing platforms: a workstation equipped with an NVIDIA Titan X Pascal and an embedded system based on an NVIDIA Jetson TX1 board. In addition, on the Jetson Nano embedded edge computing platform, different inference frameworks are investigated to evaluate the inference efficiency of the DNN models. Regression models are established to characterize the variation in the computing performance of different DNN classification algorithms so that the inference results of unknown models can be estimated. ANOVA methods are proposed to quantify the differences between models. The experimental results have important guiding significance for the better selection, deployment and application of DNN models in practice. Codes are available at this https URL https://github.com/Foreverzfy/Model-Test.", "Unmanned Aerial Vehicles (UAVs), which can operate autonomously in dynamic and complex environments, are becoming increasingly common. Deep learning techniques for motion control have recently taken a major qualitative step since vision-based inference tasks can be executed directly on edge. The goal is to fully integrate the machine learning (ML) element into small UAVs. However, given the limited payload capacity and energy available on small UAVs, integrating computing resources sufficient to host ML and vehicle control functions is still challenging. This paper presents a modular and generic system that can control the UAV by evaluating vision-based ML tasks directly inside the resource-constrained UAV. Two different vision-based navigation configurations were tested and demonstrated. The first configuration implements an autonomous landing site detection system, tested with two models based on LeNet-5 and MobileNetV2, respectively. This allows the UAV to change its planned path accordingly and approach the target to land. Moreover, a model for people detection based on a custom MobileNetV2 network was evaluated in the second configuration. Finally, the execution time and power consumption were measured and compared with a cloud computing approach. The results show the ability of the developed system to dynamically react to the environment to provide the necessary maneuver after detecting the target exploiting only the constrained computational resources of the UAV controller. Furthermore, we demonstrated that moving to the edge, instead of using cloud computing inference, decreases the energy requirement of the system without reducing the quality of service.", "With the continuous development of Internet of Things (IoT) and the overwhelming explosion of Big Data, edge computing serves as an efficient computing mode for time stringent data processing, which can bypass the constraints of network bandwidth and delay, and has been one of the foundation of interconnected applications. Although edge computing has gradually become one of bridges between cloud computing centers and mobile terminals, the literature still lacks a thorough review on the recent advances in edge computing platforms. In this paper, we firstly introduce the definition of edge computing and advantages of edge computing platform. And then, we summarize the key technologies of constructing an edge computing platform, and propose a general framework for edge computing platform. The role of distributed storage management systems in building edge computing platform is elaborated in detail. Furthermore, we give some applications to illustrate how to use third-party edge computing platforms to build specific applications. Finally, we briefly outline current open issues of edge computing platform based on our literature survey."]
            
            # Встроенный список смежности (обратите внимание: ключи-строки; если нужно int — можно привести позже)
            adj = {"0": [1, 3], "1": [0, 2, 4], "2": [1, 3, 5, 6], "3": [0, 2, 8], "4": [1, 10, 11], "5": [2, 8, 10], "6": [2, 7], "7": [6, 12], "8": [3, 5, 7, 9], "9": [8, 10], "10": [4, 5, 9, 11, 16], "11": [4, 10, 13], "12": [7, 14], "13": [11, 14], "14": [12, 13, 15], "15": [14, 16], "16": [10, 15, 17], "17": [16, 18], "18": [17, 19], "19": [18]}
    except Exception as e:

        # Любая ошибка парсинга DOCX — не фейлим ноутбук, а мягко используем embedded
        print("[WARN] DOCX parsing failed:", e, "\nFallback to embedded.")

        documents = ["Streaming data is the data from sensors as well as other real-time surveillance systems. Distributed stream processing systems are the software that manages such data. Such frameworks have to deliver outcomes on the go instantly. They are susceptible to delay and malfunction or system failures. The system must be tolerant of faults and always accessible. Many variables, such as improved network arrival rates, node failures, and so on, disrupt the system's reliability. Some operators need to be relocated online from one physical resource to another to manage or reimburse a slow or failing node. In this study, we propose a co-location based systematic migration heuristic for live operator migration between physical resources using a migration map revised with costs for each migration. The suggested method evaluates continuous operator performance patterns and makes online scheduling decisions based on the same. The decisions include migrating operators during a node failure or straggling.", "Distributed stream processing engines are designed with a focus on scalability to process big data volumes in a continuous manner. We present the Theodolite method for benchmarking the scalability of distributed stream processing engines. Core of this method is the definition of use cases that microservices implementing stream processing have to fulfill. For each use case, our method identifies relevant workload dimensions that might affect the scalability of a use case. We propose to design one benchmark per use case and relevant workload dimension. We present a general benchmarking framework, which can be applied to execute the individual benchmarks for a given use case and workload dimension. Our framework executes an implementation of the use case's dataflow architecture for different workloads of the given dimension and various numbers of processing instances. This way, it identifies how resources demand evolves with increasing workloads. Within the scope of this paper, we present 4 identified use cases, derived from processing Industrial Internet of Things data, and 7 corresponding workload dimensions. We provide implementations of 4 benchmarks with Kafka Streams and Apache Flink as well as an implementation of our benchmarking framework to execute scalability benchmarks in cloud environments. We use both for evaluating the Theodolite method and for benchmarking Kafka Streams' and Flink's scalability for different deployment options.", "Batch and stream processing are separately and efficiently applied in many applications. However, some newer data-driven applications such as the Internet of Things and cloud computing call for hybrid processing approaches in order to handle the speed and accuracy required for processing such complex data. In this paper, we propose a Hybrid Distributed Batch-Stream (HDBS) architecture for anomaly detection in real-time data. The hybrid architecture, while benefiting from the accuracy provided by batch processing, also enjoys the speed and real-time features of stream processing. In the proposed architecture, our focus is on the algorithmic aspects of hybrid processing including the interaction models between batch and stream processing units, the characteristics of batch and stream machine learning algorithms and the principles of merging the results of different processing units. The driving idea of such combination is that the results of batch and stream processing units are complementary with each other, as one of them constructs accurate models based on previous data, and the other one is capable of processing new stream data in real-time. Furthermore, we propose a generalized version of the HDBS with respect to its algorithms and communication policy levels. In the generalized HDBS architecture, we address the various aspects of the interaction between the batch and stream processing units, and the merging operations to produce the final results. the evaluations of the proposed architecture using various criteria (accuracy, space complexity, and time complexity) demonstrate that the accuracy of the proposed method is higher than the accuracy of the batch processing methods, its time complexity is also similar to one of the stream processing methods and much less than the batch processing methods, which makes our proposed architecture an efficient and practical solution for real-time anomaly detection.", "There have been increasing demands for real time processing of the ever-growing data. In order to meet this requirement and ensure the reliable processing of streaming data, a variety of distributed stream processing architectures and platforms have been developed, which handles the fundamental task of allocating processing tasks to the currently available physical resources and routing streaming data between these resources. However, many stream processing systems lack an intelligent scheduling mechanism, in which their default schedulers allocate tasks without taking resource demands and availability, or the transfer latency between resources into consideration. Besides, stream processing has a strict request for latency. Thus it is important to give latency guarantee for distributed stream processing. In this paper, we propose two new algorithms for stream processing with latency guarantee, both the algorithms consider transfer latency and resource demand in task allocation. Both algorithms can guarantee latency constraints. Algorithm AHA reduces more than 21.3% and 58.9% resources compared with the greedy and the round-robin algorithms, and algorithm PHA further improves the resource utilization to 32.1% and 73.2%.", "In the era of Big Data, typical architecture of distributed real-time stream processing systems is the combination of Flume, Kafka, and Storm. As a kind of distributed message system, Kafka has the characteristics of horizontal scalability and high throughput, which is manly deployed in many areas in order to address the problem of speed mismatch between message producers and consumers. When using Kafka, we need to quickly receive data sent by producers. In addition, we need to send data to consumers quickly. Therefore, the performance of Kafka is of critical importance to the performance of the whole stream processing system. In this paper, we propose the improved design of real-time stream processing systems, and focus on improving the Kafka’s data loading process. We use Kafka cat to transfer data from the source to Kafka topic directly, which can reduce the network transmission. We also utilize the memory file system to accelerate the process of data loading, which can address the bottleneck and performance problems caused by disk I/O. Extensive experiments are conducted to evaluate the performance, which show the superiority of our improved design.", "In this paper, nearly 40 commonly used deep neural network(DNN) models are selected, and their cross-platform and cross-inference frameworks are deeply analysed. The main metrics of accuracy, the total number of model parameters, the computational complexity, the accuracy density, the inference time, the memory consumption and other related parameters are used to measure their performance. The heterogeneous computing experiment is implemented on both the Google Colab cloud computing platform and the Jetson Nano embedded edge computing platform. The obtained performance is compared with that of two previous computing platforms: a workstation equipped with an NVIDIA Titan X Pascal and an embedded system based on an NVIDIA Jetson TX1 board. In addition, on the Jetson Nano embedded edge computing platform, different inference frameworks are investigated to evaluate the inference efficiency of the DNN models. Regression models are established to characterize the variation in the computing performance of different DNN classification algorithms so that the inference results of unknown models can be estimated. ANOVA methods are proposed to quantify the differences between models. The experimental results have important guiding significance for the better selection, deployment and application of DNN models in practice. Codes are available at this https URL https://github.com/Foreverzfy/Model-Test.", "Unmanned Aerial Vehicles (UAVs), which can operate autonomously in dynamic and complex environments, are becoming increasingly common. Deep learning techniques for motion control have recently taken a major qualitative step since vision-based inference tasks can be executed directly on edge. The goal is to fully integrate the machine learning (ML) element into small UAVs. However, given the limited payload capacity and energy available on small UAVs, integrating computing resources sufficient to host ML and vehicle control functions is still challenging. This paper presents a modular and generic system that can control the UAV by evaluating vision-based ML tasks directly inside the resource-constrained UAV. Two different vision-based navigation configurations were tested and demonstrated. The first configuration implements an autonomous landing site detection system, tested with two models based on LeNet-5 and MobileNetV2, respectively. This allows the UAV to change its planned path accordingly and approach the target to land. Moreover, a model for people detection based on a custom MobileNetV2 network was evaluated in the second configuration. Finally, the execution time and power consumption were measured and compared with a cloud computing approach. The results show the ability of the developed system to dynamically react to the environment to provide the necessary maneuver after detecting the target exploiting only the constrained computational resources of the UAV controller. Furthermore, we demonstrated that moving to the edge, instead of using cloud computing inference, decreases the energy requirement of the system without reducing the quality of service.", "With the continuous development of Internet of Things (IoT) and the overwhelming explosion of Big Data, edge computing serves as an efficient computing mode for time stringent data processing, which can bypass the constraints of network bandwidth and delay, and has been one of the foundation of interconnected applications. Although edge computing has gradually become one of bridges between cloud computing centers and mobile terminals, the literature still lacks a thorough review on the recent advances in edge computing platforms. In this paper, we firstly introduce the definition of edge computing and advantages of edge computing platform. And then, we summarize the key technologies of constructing an edge computing platform, and propose a general framework for edge computing platform. The role of distributed storage management systems in building edge computing platform is elaborated in detail. Furthermore, we give some applications to illustrate how to use third-party edge computing platforms to build specific applications. Finally, we briefly outline current open issues of edge computing platform based on our literature survey."]
        
        # Тот же embedded-граф на случай исключения при парсинге DOCX
        adj = {"0": [1, 3], "1": [0, 2, 4], "2": [1, 3, 5, 6], "3": [0, 2, 8], "4": [1, 10, 11], "5": [2, 8, 10], "6": [2, 7], "7": [6, 12], "8": [3, 5, 7, 9], "9": [8, 10], "10": [4, 5, 9, 11, 16], "11": [4, 10, 13], "12": [7, 14], "13": [11, 14], "14": [12, 13, 15], "15": [14, 16], "16": [10, 15, 17], "17": [16, 18], "18": [17, 19], "19": [18]}
else:
    # Режим embedded по умолчанию: тот же корпус/граф зашиты прямо в ноутбук
    documents = ["Streaming data is the data from sensors as well as other real-time surveillance systems. Distributed stream processing systems are the software that manages such data. Such frameworks have to deliver outcomes on the go instantly. They are susceptible to delay and malfunction or system failures. The system must be tolerant of faults and always accessible. Many variables, such as improved network arrival rates, node failures, and so on, disrupt the system's reliability. Some operators need to be relocated online from one physical resource to another to manage or reimburse a slow or failing node. In this study, we propose a co-location based systematic migration heuristic for live operator migration between physical resources using a migration map revised with costs for each migration. The suggested method evaluates continuous operator performance patterns and makes online scheduling decisions based on the same. The decisions include migrating operators during a node failure or straggling.", "Distributed stream processing engines are designed with a focus on scalability to process big data volumes in a continuous manner. We present the Theodolite method for benchmarking the scalability of distributed stream processing engines. Core of this method is the definition of use cases that microservices implementing stream processing have to fulfill. For each use case, our method identifies relevant workload dimensions that might affect the scalability of a use case. We propose to design one benchmark per use case and relevant workload dimension. We present a general benchmarking framework, which can be applied to execute the individual benchmarks for a given use case and workload dimension. Our framework executes an implementation of the use case's dataflow architecture for different workloads of the given dimension and various numbers of processing instances. This way, it identifies how resources demand evolves with increasing workloads. Within the scope of this paper, we present 4 identified use cases, derived from processing Industrial Internet of Things data, and 7 corresponding workload dimensions. We provide implementations of 4 benchmarks with Kafka Streams and Apache Flink as well as an implementation of our benchmarking framework to execute scalability benchmarks in cloud environments. We use both for evaluating the Theodolite method and for benchmarking Kafka Streams' and Flink's scalability for different deployment options.", "Batch and stream processing are separately and efficiently applied in many applications. However, some newer data-driven applications such as the Internet of Things and cloud computing call for hybrid processing approaches in order to handle the speed and accuracy required for processing such complex data. In this paper, we propose a Hybrid Distributed Batch-Stream (HDBS) architecture for anomaly detection in real-time data. The hybrid architecture, while benefiting from the accuracy provided by batch processing, also enjoys the speed and real-time features of stream processing. In the proposed architecture, our focus is on the algorithmic aspects of hybrid processing including the interaction models between batch and stream processing units, the characteristics of batch and stream machine learning algorithms and the principles of merging the results of different processing units. The driving idea of such combination is that the results of batch and stream processing units are complementary with each other, as one of them constructs accurate models based on previous data, and the other one is capable of processing new stream data in real-time. Furthermore, we propose a generalized version of the HDBS with respect to its algorithms and communication policy levels. In the generalized HDBS architecture, we address the various aspects of the interaction between the batch and stream processing units, and the merging operations to produce the final results. the evaluations of the proposed architecture using various criteria (accuracy, space complexity, and time complexity) demonstrate that the accuracy of the proposed method is higher than the accuracy of the batch processing methods, its time complexity is also similar to one of the stream processing methods and much less than the batch processing methods, which makes our proposed architecture an efficient and practical solution for real-time anomaly detection.", "There have been increasing demands for real time processing of the ever-growing data. In order to meet this requirement and ensure the reliable processing of streaming data, a variety of distributed stream processing architectures and platforms have been developed, which handles the fundamental task of allocating processing tasks to the currently available physical resources and routing streaming data between these resources. However, many stream processing systems lack an intelligent scheduling mechanism, in which their default schedulers allocate tasks without taking resource demands and availability, or the transfer latency between resources into consideration. Besides, stream processing has a strict request for latency. Thus it is important to give latency guarantee for distributed stream processing. In this paper, we propose two new algorithms for stream processing with latency guarantee, both the algorithms consider transfer latency and resource demand in task allocation. Both algorithms can guarantee latency constraints. Algorithm AHA reduces more than 21.3% and 58.9% resources compared with the greedy and the round-robin algorithms, and algorithm PHA further improves the resource utilization to 32.1% and 73.2%.", "In the era of Big Data, typical architecture of distributed real-time stream processing systems is the combination of Flume, Kafka, and Storm. As a kind of distributed message system, Kafka has the characteristics of horizontal scalability and high throughput, which is manly deployed in many areas in order to address the problem of speed mismatch between message producers and consumers. When using Kafka, we need to quickly receive data sent by producers. In addition, we need to send data to consumers quickly. Therefore, the performance of Kafka is of critical importance to the performance of the whole stream processing system. In this paper, we propose the improved design of real-time stream processing systems, and focus on improving the Kafka’s data loading process. We use Kafka cat to transfer data from the source to Kafka topic directly, which can reduce the network transmission. We also utilize the memory file system to accelerate the process of data loading, which can address the bottleneck and performance problems caused by disk I/O. Extensive experiments are conducted to evaluate the performance, which show the superiority of our improved design.", "In this paper, nearly 40 commonly used deep neural network(DNN) models are selected, and their cross-platform and cross-inference frameworks are deeply analysed. The main metrics of accuracy, the total number of model parameters, the computational complexity, the accuracy density, the inference time, the memory consumption and other related parameters are used to measure their performance. The heterogeneous computing experiment is implemented on both the Google Colab cloud computing platform and the Jetson Nano embedded edge computing platform. The obtained performance is compared with that of two previous computing platforms: a workstation equipped with an NVIDIA Titan X Pascal and an embedded system based on an NVIDIA Jetson TX1 board. In addition, on the Jetson Nano embedded edge computing platform, different inference frameworks are investigated to evaluate the inference efficiency of the DNN models. Regression models are established to characterize the variation in the computing performance of different DNN classification algorithms so that the inference results of unknown models can be estimated. ANOVA methods are proposed to quantify the differences between models. The experimental results have important guiding significance for the better selection, deployment and application of DNN models in practice. Codes are available at this https URL https://github.com/Foreverzfy/Model-Test.", "Unmanned Aerial Vehicles (UAVs), which can operate autonomously in dynamic and complex environments, are becoming increasingly common. Deep learning techniques for motion control have recently taken a major qualitative step since vision-based inference tasks can be executed directly on edge. The goal is to fully integrate the machine learning (ML) element into small UAVs. However, given the limited payload capacity and energy available on small UAVs, integrating computing resources sufficient to host ML and vehicle control functions is still challenging. This paper presents a modular and generic system that can control the UAV by evaluating vision-based ML tasks directly inside the resource-constrained UAV. Two different vision-based navigation configurations were tested and demonstrated. The first configuration implements an autonomous landing site detection system, tested with two models based on LeNet-5 and MobileNetV2, respectively. This allows the UAV to change its planned path accordingly and approach the target to land. Moreover, a model for people detection based on a custom MobileNetV2 network was evaluated in the second configuration. Finally, the execution time and power consumption were measured and compared with a cloud computing approach. The results show the ability of the developed system to dynamically react to the environment to provide the necessary maneuver after detecting the target exploiting only the constrained computational resources of the UAV controller. Furthermore, we demonstrated that moving to the edge, instead of using cloud computing inference, decreases the energy requirement of the system without reducing the quality of service.", "With the continuous development of Internet of Things (IoT) and the overwhelming explosion of Big Data, edge computing serves as an efficient computing mode for time stringent data processing, which can bypass the constraints of network bandwidth and delay, and has been one of the foundation of interconnected applications. Although edge computing has gradually become one of bridges between cloud computing centers and mobile terminals, the literature still lacks a thorough review on the recent advances in edge computing platforms. In this paper, we firstly introduce the definition of edge computing and advantages of edge computing platform. And then, we summarize the key technologies of constructing an edge computing platform, and propose a general framework for edge computing platform. The role of distributed storage management systems in building edge computing platform is elaborated in detail. Furthermore, we give some applications to illustrate how to use third-party edge computing platforms to build specific applications. Finally, we briefly outline current open issues of edge computing platform based on our literature survey."]
    adj = {"0": [1, 3], "1": [0, 2, 4], "2": [1, 3, 5, 6], "3": [0, 2, 8], "4": [1, 10, 11], "5": [2, 8, 10], "6": [2, 7], "7": [6, 12], "8": [3, 5, 7, 9], "9": [8, 10], "10": [4, 5, 9, 11, 16], "11": [4, 10, 13], "12": [7, 14], "13": [11, 14], "14": [12, 13, 15], "15": [14, 16], "16": [10, 15, 17], "17": [16, 18], "18": [17, 19], "19": [18]}

# Нормализуем ключи и соседей графа к int 
adj = {int(k): [int(x) for x in v] for k, v in adj.items()}

# Итоговый отчёт о том, что именно выбрано и сколько элементов получено
print(f"Data mode: {DATA_MODE}. Documents: {len(documents)}; Nodes: {len(adj)}")

[WARN] DOCX parsing failed: [Errno 2] No such file or directory: 'Lab1_данные.docx' 
Fallback to embedded.
Data mode: docx. Documents: 8; Nodes: 20


In [6]:
import math
from collections import defaultdict     # словари со значениями по умолч (после расчёта)

N_DOCS = len(documents)     # размер корпуса (число документов) для расчёта IDF

TOKEN_RE = re.compile(r"[A-Za-z][A-Za-z\-]+")       # Токенайзер: слова на латинице, допускаем дефис и апостроф внутри слова

def tokenize(text : str):
    return [w.lower() for w in TOKEN_RE.findall(text)]      # находим все токены по шаблону и приводим к нижнему регистру

In [7]:
result = parse_lab_data_from_docx(Path.joinpath(Path.cwd(), "resources", DOCX_PATH))
print(result)

(["Streaming data is the data from sensors as well as other real-time surveillance systems. Distributed stream processing systems are the software that manages such data. Such frameworks have to deliver outcomes on the go instantly. They are susceptible to delay and malfunction or system failures. The system must be tolerant of faults and always accessible. Many variables, such as improved network arrival rates, node failures, and so on, disrupt the system's reliability. Some operators need to be relocated online from one physical resource to another to manage or reimburse a slow or failing node. In this study, we propose a co-location based systematic migration heuristic for live operator migration between physical resources using a migration map revised with costs for each migration. The suggested method evaluates continuous operator performance patterns and makes online scheduling decisions based on the same. The decisions include migrating operators during a node failure or straggl

## TF‑IDF — Этап 1: частоты слов в документе

In [8]:
from typing import Iterator

def RECORDREADER_1():
    for doc_id, text in enumerate(documents):
        yield (doc_id, text)

def MAP_1(key, value: str):
    doc_id = key
    toks = tokenize(value)
    for tok in toks:
        yield ((doc_id, tok), 1)
    # Подсчёт total токенов на документ
    yield ((doc_id, "*CNT*"), len(toks))

def REDUCE_1(key, values: Iterator[int]):
    s = 0
    for v in values:
        s += v
    yield (key, s)

print("Выполняем этап 1 TF-IDF...")
stage1_out = list(MapReduce(RECORDREADER_1, MAP_1, REDUCE_1))
print(f"Этап 1 завершен. Получено {len(stage1_out)} записей")


Выполняем этап 1 TF-IDF...
Этап 1 завершен. Получено 899 записей


## TF‑IDF — Этап 2: в скольких документах встречается слово (DF)


In [9]:
def RECORDREADER_2():
    for kv in stage1_out:
        yield kv

def MAP_2(key, value):
    # key == (doc_id, tok) OR (doc_id, "*CNT*")
    doc_id, tok = key
    if tok == "*CNT*":
        yield (("TOTAL", doc_id), value)
    else:
        # отмечаем факт присутствия терма в документе
        yield ((tok, doc_id), 1)

def REDUCE_2(key, values: Iterator[int]):
    if key[0] == "TOTAL":
        total = 0
        for v in values:
            total += v
        yield (key, total)
    else:
        tok, doc_id = key
        cnt = 0
        for v in values:
            cnt += v
        if cnt > 0:
            yield (("DF", tok), doc_id)

print("Выполняем этап 2 TF-IDF...")
stage2_out = list(MapReduce(RECORDREADER_2, MAP_2, REDUCE_2))
print(f"Этап 2 завершен. Получено {len(stage2_out)} записей")


Выполняем этап 2 TF-IDF...
Этап 2 завершен. Получено 899 записей


## TF‑IDF — Этап 3: расчёт TF‑IDF и топ‑5 слов на документ

In [10]:
def RECORDREADER_3():
    for kv in stage1_out:
        yield kv
    for kv in stage2_out:
        yield kv

def MAP_3(key, value):
    # stage1: ((doc, tok), cnt) | ((doc, "*CNT*"), total)
    # stage2: (("TOTAL", doc), total) | (("DF", tok), doc_id)
    if isinstance(key, tuple) and len(key) == 2 and isinstance(key[0], int):
        doc_id, tok = key
        if tok == "*CNT*":
            yield (("TOTAL", doc_id), value)
        else:
            yield (("TF", doc_id, tok), ("CNT", value))
    elif key[0] == "TOTAL":
        _, doc_id = key
        yield (("TOTAL", doc_id), value)
    elif key[0] == "DF":
        _, tok = key
        yield (("DFTOK", tok), value)


def REDUCE_3(group_key, values: Iterator):
    if group_key[0] == "TOTAL":
        total = 0
        for v in values:
            total += v
        yield (group_key, total)
    elif group_key[0] == "DFTOK":
        seen = set(values)
        df = len(seen)
        yield (("DF", group_key[1]), df)
    elif group_key[0] == "TF":
        cnt = 0
        for tag, v in values:
            if tag == "CNT":
                cnt += v
        yield (group_key, cnt)

print("Выполняем этап 3 TF-IDF...")
stage3_intermediate = list(MapReduce(RECORDREADER_3, MAP_3, REDUCE_3))

# Собираем TOTAL, DF, TF и считаем tf-idf
totals, dfs, tfs_raw = {}, {}, {}
for key, val in stage3_intermediate:
    if key[0] == "TOTAL":
        totals[key[1]] = val
    elif key[0] == "DF":
        dfs[key[1]] = val
    elif key[0] == "TF":
        _, doc_id, tok = key
        tfs_raw[(doc_id, tok)] = val

tfidf = defaultdict(dict)
for (doc_id, tok), tn in tfs_raw.items():
    cn = totals.get(doc_id, 1)
    tf = tn / cn if cn else 0.0
    df = dfs.get(tok, 1)
    idf = math.log(N_DOCS / df) if df > 0 else 0.0
    tfidf[doc_id][tok] = tf * idf

# Выводим топ-5 слов для каждого документа
print("\nРезультаты TF-IDF - топ-5 слов для каждого документа:")
top5_per_doc = {doc_id: sorted(scores.items(), key=lambda kv: kv[1], reverse=True)[:5]
                for doc_id, scores in tfidf.items()}

for doc_id, pairs in sorted(top5_per_doc.items()):
    print(f"Документ {doc_id}: {[(w, round(s,6)) for w,s in pairs]}")

Выполняем этап 3 TF-IDF...

Результаты TF-IDF - топ-5 слов для каждого документа:
Документ 0: [('migration', 0.028486), ('node', 0.021364), ('or', 0.01899), ('such', 0.014243), ('failures', 0.014243)]
Документ 1: [('case', 0.025114), ('benchmarking', 0.020091), ('workload', 0.020091), ('use', 0.018953), ('scalability', 0.016743)]
Документ 2: [('batch', 0.029083), ('hybrid', 0.014542), ('units', 0.014542), ('accuracy', 0.012118), ('hdbs', 0.010906)]
Документ 3: [('latency', 0.037133), ('guarantee', 0.018566), ('demands', 0.012378), ('task', 0.012378), ('resource', 0.012378)]
Документ 4: [('kafka', 0.02666), ('message', 0.011426), ('producers', 0.011426), ('consumers', 0.011426), ('quickly', 0.011426)]
Документ 5: [('dnn', 0.020794), ('jetson', 0.015596), ('embedded', 0.015596), ('models', 0.014712), ('inference', 0.013863)]
Документ 6: [('uav', 0.017474), ('uavs', 0.013106), ('control', 0.013106), ('vision-based', 0.013106), ('ml', 0.013106)]
Документ 7: [('edge', 0.029366), ('computing

## Поиск кратчайшего пути на графе с использованием алгоритма BFS

In [11]:
SRC, DST = 0, 19

def initial_graph():
    for n in sorted(adj.keys()):
        yield (n, {"adj": adj[n], "dist": (0 if n==SRC else float("inf")), "prev": (None if n!=SRC else -1)})

current_state = list(initial_graph())

def RECORDREADER_BFS():
    for kv in current_state:
        yield kv

def MAP_BFS(node_id, node_state):
    yield (node_id, ("NODE", node_state))
    d = node_state["dist"]
    if d < float("inf"):
        for nb in node_state["adj"]:
            yield (nb, ("DIST", d+1, node_id))

def REDUCE_BFS(node_id, values):
    best = {"adj": [], "dist": float("inf"), "prev": None}
    proposals = []
    for tag, *rest in values:
        if tag == "NODE":
            best = rest[0]
        elif tag == "DIST":
            proposals.append(tuple(rest))
    best_dist = best["dist"]; best_prev = best["prev"]
    for d, prev in proposals:
        if d < best_dist:
            best_dist, best_prev = d, prev
    out = {"adj": best["adj"], "dist": best_dist, "prev": best_prev}
    yield (node_id, out)

def should_stop(prev_state, new_state, target_reached: bool) -> bool:
    """Возвращает True, если нужно завершить итерации."""
    if target_reached:
        return True
    if len(prev_state) != len(new_state):
        return False
    for i in range(len(new_state)):
        if prev_state[i][1]["dist"] != new_state[i][1]["dist"] or prev_state[i][1]["prev"] != new_state[i][1]["prev"]:
            return False
    return True

def run_bfs(max_iters=200):
    global current_state
    iters = 0
    while True:
        iters += 1
        prev_state = current_state
        new_state = list(MapReduce(RECORDREADER_BFS, MAP_BFS, REDUCE_BFS))
        current_state = new_state
        target_rec = dict(current_state).get(DST, {"dist": float("inf")})
        if should_stop(prev_state, new_state, target_rec["dist"] < float("inf")) or iters >= max_iters:
            return iters, target_rec

iters, target_rec = run_bfs()
state_map = dict(current_state)
path = []
if target_rec["dist"] < float("inf"):
    node = DST
    while node is not None and node != -1:
        path.append(node)
        node = state_map[node]["prev"]
    path.reverse()

print("BFS -> iters:", iters, "distance:", target_rec["dist"], "path:", path)

BFS -> iters: 7 distance: 7 path: [0, 1, 4, 10, 16, 17, 18, 19]
